In [ ]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from concurrent.futures import ThreadPoolExecutor,ProcessPoolExecutor
from openpyxl import Workbook
import pyLDAvis
import warnings
import os
import pyLDAvis.sklearn
import pandas as pd
import jieba
warnings.filterwarnings('ignore')
df1 = pd.read_csv("data/weibo.csv", encoding='utf-8')
df=pd.DataFrame(df1['text'].astype(str))

#加载停用词表
a=r'停用词表stopwords\746.txt'
b=r'停用词表stopwords\902.txt'
c=r'停用词表stopwords\1208.txt'
d=r'停用词表stopwords\1447.txt'
e=r'停用词表stopwords\1893.txt'
f=r'停用词表stopwords\中文.txt'
g=r'停用词表stopwords\哈工大停用词表.txt'
h=r'停用词表stopwords\四川大学机器智能实验室停用词库.txt'
i=r'停用词表stopwords\百度停用词列表.txt'
j=r'停用词表stopwords\符号.txt'
k=r'停用词表stopwords\英文.txt'
l=r'停用词表stopwords\stopwords.txt'
lis=[a,b,c,d,e,f,g,h,i,j,k,l]
if os.path.exists('data/单个文本主题.xlsx'):
    os.remove('data/单个文本主题.xlsx')

def loadstop_words(path):
    # 停用词：结合多个停用词表
    stop_words = set()
    for path in lis:
        with open(path,'r',encoding='utf8') as fr:
            for line in fr:
                item = line.strip()
                stop_words.add(item)
    custom_stop_words=set()
    stop_words=stop_words | custom_stop_words
    return stop_words
stopwords=loadstop_words(lis)

#分词，并添加限制条件分词
def chinese_word_cut(text):
    s=list(jieba.cut(text))
    s1=[]
    for word in s:
        if len(word) <= 2 and word not in stopwords and not word.isdigit():
            s1.append(word)
    if not s1:
        return '空'
    else:
        return ' '.join(s1)  
df['text_cutted']=df.text.apply(chinese_word_cut)
# print(df.text_cutted)
# print(df.iloc[[1]])
lens=len(df['text_cutted'])
n_features = 1000    
n_topics = 1
n_top_words = 5

def save_topic(model, feature_names, n_top_words ,topid):
    wb = Workbook()
    ws = wb.active
    ws.append(['topic','words'])
    for topic_idx, topic in enumerate(model.components_):
        print("第%d条短文本主题:" % (topid+1))
        print(" ".join([feature_names[i]
                for i in topic.argsort()[:-n_top_words - 1:-1]]))
        line=[topid+1 ," ".join([feature_names[i]for i in topic.argsort()[:-n_top_words - 1:-1]])]
        ws.append(line)  # 将数据需要保存的项以行的形式添加到xlsx中
        wb.save(r'D:\pycharm\爬虫\data\topic.xlsx')        
    print()
tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                max_features=n_features,
                                stop_words=stopwords,
                                max_df = 5,
                                min_df = 1)

lda = LatentDirichletAllocation(n_topics=n_topics, max_iter=50,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)
def oneto():
    i=0
    while i<lens:
        try:
            df1=df['text_cutted'].iloc[[i]]   
            tf= tf_vectorizer.fit_transform(df1)
            tf_feature_names = tf_vectorizer.get_feature_names()
            lda.fit(tf)
            save_topic(lda, tf_feature_names, n_top_words,i)
            i=i+1       
        except ValueError as e:
    #         print("第%d条短文本为特殊文本无主题(如仅包含停用词):" % (i+1))
            i=i+1
            continue
    return
# executor=ProcessPoolExecutor(100)
# executor.submit(oneto)
oneto()
#整体提取
n_features =len(df.text_cutted)
n_topics = 20
n_top_words1 = 15
tf1_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                max_features=n_features,
                                stop_words=stopwords,
                                max_df = 0.5,
                                min_df = 2)
# tf1_feature_names = tf1_vectorizer.get_feature_names()
tf1 = tf1_vectorizer.fit_transform(df.text_cutted)
lda1 = LatentDirichletAllocation(n_topics=n_topics, max_iter=50,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)
lda1.fit(tf1)
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic%d:" % topic_idx)
        print(" ".join([feature_names[i]
                for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()
# print_top_words(lda, tf_feature_names, n_top_words1)

data = pyLDAvis.sklearn.prepare(lda1, tf1, tf1_vectorizer)
pyLDAvis.show(data)

第1条短文本主题:
预售 岁月 喜欢 回头 本书

第2条短文本主题:
机会 上次 路线 钢琴 艺术

第3条短文本主题:
出门 主子 感受 日常 影子

第4条短文本主题:
真好

第6条短文本主题:
姿势 一架 记得 记仇 呼噜

第8条短文本主题:
最右

第9条短文本主题:
烧烤

第10条短文本主题:
十一 不信 仓鼠 颜值 小麦

第11条短文本主题:
转眼 收到 粽子

第12条短文本主题:
劝醒 姐妹

第13条短文本主题:
领域 擅长 邂逅

第14条短文本主题:
计划 失败 恋爱

第15条短文本主题:
钢琴 朗朗 复课 妻子 羡慕

第16条短文本主题:
抖森 同框 天猫

第17条短文本主题:
赶紧 快乐 接收

第18条短文本主题:
办法 不见 耗在 说服 永远

第19条短文本主题:
丰盛 速来

第20条短文本主题:
狗狗 路上

第21条短文本主题:
恭喜

第22条短文本主题:
一条 超全

第23条短文本主题:
老师 技术 整容 这位

第24条短文本主题:
防晒 晒黑 卖得 平价 跟风

第25条短文本主题:
钢丝 护理 姐妹 安利 粗糙

第26条短文本主题:
北京 去世 世园 好吃 请问

第27条短文本主题:
帖子 夏天 小吃 问问 续命

第28条短文本主题:
心动 公交 形状 看过 玉玺

第29条短文本主题:
套上 出门 外卖 睡裙 安利

第30条短文本主题:
北极 兔界 逗笑 动图 瞬间

第31条短文本主题:
有心

第32条短文本主题:
记得 还记

第33条短文本主题:
平复 久久 妹子 心情 微信

第36条短文本主题:
傻纸

第37条短文本主题:
海豹

第40条短文本主题:
超级

第43条短文本主题:
知识

第44条短文本主题:
相爱 始终 孤独 陪伴

第45条短文本主题:
倒闭 款上

第47条短文本主题:
真香

第49条短文本主题:
调包 公司 厉害

第51条短文本主题:
奖励 两箱 配合 今日 罐头

第52条短文本主题:
假期

第55条短文本主题:
喜欢 嘻嘻 一首 拍摄 阿初

第56条短文本主题:
夕阳 无限

第57条短文本主题:
大风 太阳

第58条短文本主题:
吹风 太阳

第60条短文本主题:
钥匙 室友 不到 出门 昨天

第61条短文本主题:
想法 一件 私密 

127.0.0.1 - - [04/Jun/2019 21:23:17] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [04/Jun/2019 21:23:17] "GET /LDAvis.css HTTP/1.1" 200 -
127.0.0.1 - - [04/Jun/2019 21:23:17] "GET /d3.js HTTP/1.1" 200 -
127.0.0.1 - - [04/Jun/2019 21:23:17] "GET /LDAvis.js HTTP/1.1" 200 -
